In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from __future__ import print_function, division
import scipy
# from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, model_from_json
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
from PIL import Image

Using TensorFlow backend.


In [0]:
class CycleGAN():
  def __init__(self):
    # Input shape
    self.img_rows = 128
    self.img_cols = 128
    self.channels = 3
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.dir_path = './' + 'drive' + '/' + 'My Drive/' + '授業資料/' + 'M1/' + 'Pattern_Recognision_and_Machine_Learning/' + '後半/' + 'cifar-10/' + 'cifar-10/'

    # Calculate output shape of D (PatchGAN)
    self.patch = int(self.img_rows / 2**4)
    self.disc_patch = (patch, patch, 1)

    # Number of filters in the first layer of G and D
    self.gf = 32
    self.df = 64

    # Loss weights
    self.lambda_cycle = 10.0                    # Cycle-consistency loss
    self.lambda_id = 0.1 * self.lambda_cycle    # Identity loss

    optimizer = Adam(0.0002, 0.5)

    # Build and compile the discriminators
    self.d_A = self.build_discriminator()
    self.d_B = self.build_discriminator()
    self.d_A.compile(loss='mse',
        optimizer=optimizer,
        metrics=['accuracy'])
    self.d_B.compile(loss='mse',
        optimizer=optimizer,
        metrics=['accuracy'])

    #-------------------------
    # Construct Computational
    #   Graph of Generators
    #-------------------------

    # Build the generators
    self.g_AB = self.build_generator()
    self.g_BA = self.build_generator()

    # Input images from both domains
    img_A = Input(shape=self.img_shape)
    img_B = Input(shape=self.img_shape)

    # Translate images to the other domain
    fake_B = self.g_AB(img_A)
    fake_A = self.g_BA(img_B)
    # Translate images back to original domain
    reconstr_A = self.g_BA(fake_B)
    reconstr_B = self.g_AB(fake_A)
    # Identity mapping of images
    img_A_id = self.g_BA(img_A)
    img_B_id = self.g_AB(img_B)

    # For the combined model we will only train the generators
    self.d_A.trainable = False
    self.d_B.trainable = False

    # Discriminators determines validity of translated images
    valid_A = self.d_A(fake_A)
    valid_B = self.d_B(fake_B)

    # Combined model trains generators to fool discriminators
    self.combined = Model(inputs=[img_A, img_B],
                          outputs=[ valid_A, valid_B,
                                    reconstr_A, reconstr_B,
                                    img_A_id, img_B_id ])
    self.combined.compile(loss=['mse', 'mse',
                                'mae', 'mae',
                                'mae', 'mae'],
                        loss_weights=[  1, 1,
                                        self.lambda_cycle, self.lambda_cycle,
                                        self.lambda_id, self.lambda_id ],
                        optimizer=optimizer)

  def build_generator(self):
    """U-Net Generator"""
    
    def conv2d(layer_input, filters, f_size=4):
      """Layers used during downsampling"""
      d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
      d = LeakyReLU(alpha=0.2)(d)
  #         d = InstanceNormalization()(d)
      return d

    def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
      """Layers used during upsampling"""
      u = UpSampling2D(size=2)(layer_input)
      u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
      if dropout_rate:
        u = Dropout(dropout_rate)(u)
      #       u = InstanceNormalization()(u)
      u = Concatenate()([u, skip_input])
      return u

    # Image input
    d0 = Input(shape=self.img_shape)
    # Downsampling
    d1 = conv2d(d0, self.gf)    
    d2 = conv2d(d1, self.gf*2)    
    d3 = conv2d(d2, self.gf*4)    
    d4 = conv2d(d3, self.gf*8)    

    # Upsampling
    u1 = deconv2d(d4, d3, self.gf*4)
    u2 = deconv2d(u1, d2, self.gf*2)
    u3 = deconv2d(u2, d1, self.gf)

    u4 = UpSampling2D(size=2)(u3)
    output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

    return Model(d0, output_img)

  def build_discriminator(self):

    def d_layer(layer_input, filters, f_size=4, normalization=True):
      """Discriminator layer"""
      d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
      d = LeakyReLU(alpha=0.2)(d)
  #         if normalization:
  #             d = InstanceNormalization()(d)
      return d

    img = Input(shape=self.img_shape)

    d1 = d_layer(img, self.df, normalization=False)
    d2 = d_layer(d1, self.df*2)
    d3 = d_layer(d2, self.df*4)
    d4 = d_layer(d3, self.df*8)

    validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

    return Model(img, validity)

  def train(self, epochs, train_nums, batch_size=1):
    valid = np.ones((train_nums,) + self.disc_patch)
    fake = np.zeros((train_nums,) + self.disc_patch)

    for epoch in range(epochs):
      imgs_A, imgs_B = self.data_read(con='train',n_data=train_nums)
      
#     Translate images to opposite domain
      fake_B = self.g_AB.predict(imgs_A)
      fake_A = self.g_BA.predict(imgs_B)

      # Train the discriminators (original images = real / translated = Fake)
      dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
      dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
      dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

      dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
      dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
      dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

      # Total disciminator loss
      d_loss = 0.5 * np.add(dA_loss, dB_loss)


      # ------------------
      #  Train Generators
      # ------------------

      # Train the generators
      g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                      [valid, valid,
                                      imgs_A, imgs_B,
                                      imgs_A, imgs_B])
      print('epochs:{0}'.format(epoch))
      print('discriminator loss:{0}'.format(d_loss))           
      print('generator loss:{0}'.format(g_loss))           

  def generate(self,data_num,load):
    r, c = 2, 3
    imgs_A, imgs_B = self.data_read(con='test',n_data=data_num)
    
    if load == True:
      dir_path_name_model = self.dir_path + 'g_AB_model.json'
      dir_path_name_weights = self.dir_path + 'g_AB_weights.h5'
      # g_ABモデルを読み込む
      g_AB = model_from_json(open(dir_path_name_model).read())
      # 学習結果を読み込む
      g_AB.load_weights(dir_path_name_weights)
#       g_AB.summary()
#       g_AB.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])
      
      dir_path_name_model = self.dir_path + 'g_BA_model.json'
      dir_path_name_weights = self.dir_path + 'g_BA_weights.h5'
      # g_BAモデルを読み込む
      g_BA = model_from_json(open(dir_path_name_model).read())
      # 学習結果を読み込む
      g_BA.load_weights(dir_path_name_weights)

    # Translate images to the other domain
    fakes_B = self.g_AB.predict(imgs_A)
    fakes_A = self.g_BA.predict(imgs_B)

    # Translate back to original domain
    reconstrs_A = self.g_BA.predict(fakes_B)
    reconstrs_B = self.g_AB.predict(fakes_A)
    
#     print('imgs_A:{0}'.format(imgs_A.shape))
#     print('fakes_B:{0}'.format(fakes_B.shape))
#     print('reconst_A:{0}'.format(reconstrs_A.shape))
    
    for i in range(data_num):
      gen_imgs = np.stack([imgs_A[i], fakes_B[i], reconstrs_A[i], imgs_B[i], fakes_A[i], reconstrs_B[i]])
      
      print('gen:{0}'.format(gen_imgs.shape))

      # Rescale images 0 - 1
      gen_imgs = 0.5 * gen_imgs + 0.5

      titles = ['Original', 'Translated', 'Reconstructed']
      fig, axs = plt.subplots(r, c)
      cnt = 0
      for j in range(r):
        for k in range(c):
#               print(gen_imgs[cnt].shape)
              axs[j,k].imshow(gen_imgs[cnt])
              axs[j, k].set_title(titles[k])
              axs[j,k].axis('off')
              cnt += 1
      gen_path_name = self.dir_path + 'result' + '/' + 'generate_' + str(i) + '.png'
      print(i)
      fig.savefig(gen_path_name)
      plt.close()

  def data_read(self,con,n_data):
    if con == "train":
        dir_path_2 = self.dir_path + 'train'
    elif con == "test":
        dir_path_2 = self.dir_path + 'test'

    imgs_A = np.zeros([n_data,self.img_rows,self.img_rows,self.channels])
    imgs_B = np.zeros([n_data,self.img_rows,self.img_rows,self.channels])

    for i in range(n_data):
      img_A = dir_path_2 + '/' + 'airplane' + '/' + str(i) + '.png'
      img_B = dir_path_2 + '/' + 'automobile' + '/' + str(i) + '.png'
      work_img_A = Image.open(img_A).convert('RGB')
      resize_img_A = work_img_A.resize((self.img_rows, self.img_rows))
      work_img_B = Image.open(img_B).convert('RGB')
      resize_img_B = work_img_B.resize((self.img_rows, self.img_rows))
      data_vec_A = np.asarray(resize_img_A).astype(np.float64)
      data_vec_B = np.asarray(resize_img_B).astype(np.float64)
#       if i == 0:
#         print('data_vec_A:{0}'.format(data_vec_A))
      data_vec_A = data_vec_A / 255.0
      data_vec_B = data_vec_B / 255.0
#       if i == 0:
#         print('data_vec_A:{0}'.format(data_vec_A.shape))
#       data_vec_B = data_vec_A / np.sum( data_vec_A )
#       data_vec_B = data_vec_B / np.sum( data_vec_B )

      imgs_A[i] = data_vec_A
      imgs_B[i] = data_vec_B
    
    return imgs_A, imgs_B

  def save_model(self):
    # モデルとパラメタを別々に保存
    dir_path_name_model = self.dir_path + 'g_AB_model.json'
    dir_path_name_weights = self.dir_path + 'g_AB_weights.h5'
    g_AB_json = self.g_AB.to_json()
    open(dir_path_name_model, 'w').write(g_AB_json)
    self.g_AB.save_weights(dir_path_name_weights)
    
    dir_path_name_model = self.dir_path + 'g_BA_model.json'
    dir_path_name_weights = self.dir_path + 'g_BA_weights.h5'
    g_BA_json = self.g_BA.to_json()
    open(dir_path_name_model, 'w').write(g_BA_json)
    self.g_BA.save_weights(dir_path_name_weights)
gan = CycleGAN()

In [12]:
# print(gan.g_AB.summary())
gan.train(epochs=100, train_nums=100, batch_size=1)

W0820 02:49:14.767649 140054762952576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0820 02:49:14.781833 140054762952576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epochs:0
discriminator loss:[0.4868436 0.5      ]
generator loss:[13.102646, 0.99825865, 0.97831094, 0.5465702, 0.4503237, 0.68185335, 0.4752836]
epochs:1
discriminator loss:[0.14851294 0.75109375]
generator loss:[12.450613, 1.1244599, 1.1442883, 0.49020985, 0.41987824, 0.6171743, 0.46380994]
epochs:2
discriminator loss:[0.6890434  0.67636716]
generator loss:[9.7731, 1.1599498, 1.0384514, 0.3058384, 0.36759537, 0.43137607, 0.40898427]
epochs:3
discriminator loss:[0.3523541 0.5020703]
generator loss:[11.226782, 0.66909736, 0.9930671, 0.34335232, 0.5305711, 0.30523738, 0.52014554]
epochs:4
discriminator loss:[0.4030725  0.35242188]
generator loss:[8.538177, 0.9104491, 0.5425303, 0.38183373, 0.25935757, 0.4144882, 0.2587966]
epochs:5
discriminator loss:[0.2405057 0.5788281]
generator loss:[7.359299, 0.90901303, 0.8925861, 0.23617052, 0.24377173, 0.4736966, 0.28458083]
epochs:6
discriminator loss:[0.1792547 0.7081641]
generator loss:[7.587137, 0.99103624, 0.9657358, 0.25667036, 0.23424362,

In [13]:
gan.save_model()
gan.generate(data_num=10,load=True)

gen:(6, 128, 128, 3)
0
gen:(6, 128, 128, 3)
1
gen:(6, 128, 128, 3)
2
gen:(6, 128, 128, 3)
3
gen:(6, 128, 128, 3)
4
gen:(6, 128, 128, 3)
5
gen:(6, 128, 128, 3)
6
gen:(6, 128, 128, 3)
7
gen:(6, 128, 128, 3)
8
gen:(6, 128, 128, 3)
9


In [0]:
gan.g_AB.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 64, 64, 32)   1568        input_9[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_25 (LeakyReLU)      (None, 64, 64, 32)   0           conv2d_37[0][0]                  
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 32, 32, 64)   32832       leaky_re_lu_25[0][0]             
__________________________________________________________________________________________________
leaky_re_l